<a href="https://colab.research.google.com/github/WaGi-Coding/TakiBark_Colab-Notebook/blob/main/TakiBark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About

This is a Notebook to generate [Text-To-Speech(TTS)](https://en.wikipedia.org/wiki/Speech_synthesis) Audio with AI using [Bark](https://github.com/suno-ai/bark).<br>
<br>
I initially made it for myself to make it easier to use, picking a language setting a seed, tune the setting and so on.
<br><br>
#### Creator:<br>
* **GitHub:** [WaGi-Coding](https://github.com/WaGi-Coding)
* **Discord:** Taki7o7#1337
<br><br>

#### The Code-Thief has stolen code from following GitHub Repos:
*   [suno-ai/bark](https://github.com/suno-ai/bark/)
*   [C0untFloyd/bark-gui](https://github.com/C0untFloyd/bark-gui)





# RUN SETUP - This will take a while

**↓ RUN HERE**<br>



In [ ]:
#@title RUN THIS SETUP!

! pip install pytorch_seed

! git clone https://github.com/suno-ai/bark
! cd bark && pip install .

import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio, clear_output
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
    _load_history_prompt,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE
from typing import Dict, Optional, Union
import numpy as np
from bark.generation import codec_decode, generate_coarse, generate_fine, generate_text_semantic

# From: https://github.com/C0untFloyd/bark-gui/blob/main/bark/api.py
def generate_with_settings(text_prompt, semantic_temp=0.6, eos_p=0.2, coarse_temp=0.7, fine_temp=0.5, voice_name=None, output_full=False):

    # generation with more control
    x_semantic = generate_text_semantic(
        text_prompt,
        history_prompt=voice_name,
        temp=semantic_temp,
        min_eos_p = eos_p,
        use_kv_caching=True
    )

    x_coarse_gen = generate_coarse(
        x_semantic,
        history_prompt=voice_name,
        temp=coarse_temp,
        use_kv_caching=True
    )
    x_fine_gen = generate_fine(
        x_coarse_gen,
        history_prompt=voice_name,
        temp=fine_temp,
    )

    if output_full:
        full_generation = {
            'semantic_prompt': x_semantic,
            'coarse_prompt': x_coarse_gen,
            'fine_prompt': x_fine_gen
        }
        return full_generation, codec_decode(x_fine_gen)
    return codec_decode(x_fine_gen)


nltk.download('punkt')

In [ ]:
#@title RUN THIS to preload the Models
#@markdown Takes some time :)
preload_models()

# $ \\ $

# Generation

In [ ]:
#@title Run this & select a language.
#@markdown <b>Just a sidenote if you open the notebook later again and still see the output:</b><br>You always have to run it once, in order to register the observer which keeps track of the changes.

from IPython import display
import ipywidgets as widgets
import os

def InitLangLangOptions():
    # 3rd parameter is an array of indexes which are female
    # 4th parameter is a dictionary with indexes optional additional tag
    LANGS = [
        ("English", "en", [9], {5: "Grainy", 6: "Suno Favorite"}),
        ("German", "de", [3, 8], {3: "Taki's Favorite"}),
        ("Spanish", "es", [8, 9], {2: "BG Noise", 3: "BG Noise", 5: "BG Noise"}),
        ("French", "fr", [1, 2, 5], {9: "Auditorium"}),
        ("Hindi", "hi", [0, 1, 3, 4, 9], {1: "BG Noise", 4: "BG Noise"}),
        ("Italian", "it", [2, 7, 9], {4: "Suno Favorite"}),
        ("Japanese", "ja", [0, 1, 3, 4, 5, 7, 8, 9], {1: "BG Noise"}),
        ("Korean", "ko", [0], {}),
        ("Polish", "pl", [4, 6, 9], {}),
        ("Portuguese", "pt", [], {6: "BG Noise"}),
        ("Russian", "ru", [5, 6, 9], {1: "Echoes", 2: "Echoes", 6: "Grainy", 8: "Grainy", 9: "Grainy"}),
        ("Turkish", "tr", [4, 5], {7: "Grainy"}),
        ("Chinese", "zh", [4, 6, 7, 9], {6: "BG Noise"}),
    ]
    
    # Create the category dropdown widget
    languages_dropdown = widgets.Dropdown(
        options=[lang[0] for lang in LANGS] + ["Misc."],
        description='Language:',
        layout=widgets.Layout(width='250px')
    )
    
    # Create the gender dropdown widget
    gender_dropdown = widgets.Dropdown(
        options=["All", "Male", "Female"],
        description='Gender:',
        layout=widgets.Layout(width='250px')
    )
    
    # Create the options dropdown widget
    options_dropdown = widgets.Dropdown(
        description='Options:',
        layout=widgets.Layout(width='350px')
    )
    
    # Function to update the options dropdown based on the selected category and gender
    def update_options_dropdown(change):
        selected_category = languages_dropdown.value
        selected_gender = gender_dropdown.value
    
        if selected_category == "Misc.":
            options_dropdown.options = [("announcer", "announcer")]
        else:
            lang = next((lang for lang in LANGS if lang[0] == selected_category), None)
            if lang:
                if selected_gender == "All":
                    options_dropdown.options = [
                        (f"v2{os.path.sep}{lang[1]}_speaker_{i}" + (f" | {lang[3].get(i)}" if i in lang[3] else ""), f"v2{os.path.sep}{lang[1]}_speaker_{i}")
                        for i in range(10)]
                elif selected_gender == "Male":
                    options_dropdown.options = [
                        (f"v2{os.path.sep}{lang[1]}_speaker_{i}" + (f" | {lang[3].get(i)}" if i in lang[3] else ""), f"v2{os.path.sep}{lang[1]}_speaker_{i}")
                        for i in range(10) if i not in lang[2]]

    
                elif selected_gender == "Female":
                    options_dropdown.options = [
                        (f"v2{os.path.sep}{lang[1]}_speaker_{i}" + (f" | {lang[3].get(i)}" if i in lang[3] else ""), f"v2{os.path.sep}{lang[1]}_speaker_{i}")
                        for i in lang[2]]
            else:
                options_dropdown.options = []
    
    
    # Register the function to be called when the category dropdown value changes
    languages_dropdown.observe(update_options_dropdown, names='value')
    
    # Register the function to be called when the gender dropdown value changes
    gender_dropdown.observe(update_options_dropdown, names='value')
    
    # Create an HBox widget container to horizontally arrange the dropdowns
    dropdown_container = widgets.HBox([languages_dropdown, gender_dropdown, options_dropdown])
    
    # Call the update_options_dropdown function initially to populate the options dropdown
    update_options_dropdown({'new': languages_dropdown.value})
    languages_dropdown.value = "English"
    gender_dropdown.value = "All"
    options_dropdown.value = "v2/en_speaker_9"
    # Display the dropdown widgets in the container
    display.display(dropdown_container)
    
    return languages_dropdown, gender_dropdown, options_dropdown

languages_dropdown, gender_dropdown, options_dropdown = InitLangLangOptions()

# Long-Form Generation
Because Bark works best with Text which would take ~14 seconds as it would become a weird mess if it's much longer:
<br>
We split longer text into sentences using `nltk` and generate the sentences one by one.
<br><br>
We will then combine the generated audio of each sentence together and output the full audio.

In [ ]:
#@markdown Run this once to show the Textarea
textarea = widgets.Textarea(
    value='',
    placeholder='Input your Text here!',
    
    layout=widgets.Layout(width='600px', height='150px'),
    style={'resize': 'both'}
)

display.display(textarea)

In [ ]:
#@title RUN THIS TO START GENERATING
import pytorch_seed

text = textarea.value
script = text.replace("\n", " ").strip()
sentences = nltk.sent_tokenize(script)

# seed <= 0 will be Radnom
seed = 0 #@param {type:"integer"}

# default 0.6
text_temp = 0.6 #@param {type:"number"}

# default 0.7
waveform_temp = 0.7 #@param {type:"number"}

# default 0.05
eos_prob = 0.05 #@param {type:"number"}

# default 0.25
sentence_end_silence = 0.25 #@param {type:"number"}

currentseed = seed
if seed != None and seed > 2**32 - 1:
    print(f"Seed {seed} > 2**32 - 1 (max), setting to random")
    currentseed = None
if currentseed == None or currentseed <= 0:
    currentseed = np.random.default_rng().integers(1, 2**32 - 1)

print(f"Selected speaker: {options_dropdown.value}");
print(f"Seed: {currentseed}")

silence = np.zeros(int(sentence_end_silence * SAMPLE_RATE))  # quarter second of silence

print(f"Sentence-Count: {len(sentences)}")

pieces = []
csIdx = 1
with pytorch_seed.SavedRNG(currentseed):
  for sentence in sentences:
    print(f"Generating Sentence {csIdx}/{len(sentences)}")
    audio_array = generate_with_settings(text_prompt=sentence, voice_name=options_dropdown.value, semantic_temp=text_temp, coarse_temp=waveform_temp, eos_p=eos_prob)

    pieces += [audio_array, silence.copy()]
    csIdx += 1


Selected speaker: v2/en_speaker_9
Seed: 611650384
Sentence-Count: 3
Generating Sentence 1/3


100%|██████████| 22/22 [00:21<00:00,  1.01it/s]


Generating Sentence 2/3


100%|██████████| 12/12 [00:11<00:00,  1.03it/s]


Generating Sentence 3/3


100%|██████████| 18/18 [00:18<00:00,  1.02s/it]


In [ ]:
#@markdown <--- Generate the Audio from the sentence pieces
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

# **Simple SHORT Generation**

In [ ]:
#@title RUN THIS TO START GENERATING

import pytorch_seed

text = "Hey - I am Taki7o7 and this is my custom Bark Notebook." #@param {type:"string"}

# seed <= 0 will be Radnom
seed = 0 #@param {type:"integer"}

# default 0.6
text_temp = 0.6 #@param {type:"number"}

# default 0.7
waveform_temp = 0.7 #@param {type:"number"}

# default 0.05
eos_prob = 0.05 #@param {type:"number"}


currentseed = seed
if seed != None and seed > 2**32 - 1:
    print(f"Seed {seed} > 2**32 - 1 (max), setting to random")
    currentseed = None
if currentseed == None or currentseed <= 0:
    currentseed = np.random.default_rng().integers(1, 2**32 - 1)

print(f"Selected speaker: {options_dropdown.value}");
print(f"Seed: {currentseed}")

with pytorch_seed.SavedRNG(currentseed):
  audio_array = generate_with_settings(text_prompt=text, voice_name=options_dropdown.value, semantic_temp=text_temp, coarse_temp=waveform_temp, eos_p=eos_prob)

Audio(audio_array, rate=SAMPLE_RATE)

Selected speaker: v2/en_speaker_9
Seed: 611650384


100%|██████████| 22/22 [00:21<00:00,  1.03it/s]
